# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 73 new papers today
          26 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/25 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2301.08744


/tmp/ipykernel_2084/4030337529.py:34: LatexWarning: 2301.08744 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2301.08761


extracting tarball to tmp_2301.08761...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.09614


extracting tarball to tmp_2301.09614...

 done.
Retrieving document from  https://arxiv.org/e-print/2301.09621


extracting tarball to tmp_2301.09621...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.08770-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.08770) | **JWST/MIRI Spectroscopy of the Disk of the Young Eruptive Star EX Lup in  Quiescence**  |
|| Ágnes Kóspál, et al. -- incl., <mark>Lindsey Diehl</mark>, <mark>Jeroen Bouwman</mark>, <mark>Joel D. Green</mark>, <mark>Thomas Henning</mark> |
|*Appeared on*| *2023-01-24*|
|*Comments*| *9 pages, 4 figures, accepted for publication in ApJL*|
|**Abstract**| EX Lup is a low-mass pre-main sequence star that occasionally shows accretion-related outbursts. Here, we present JWST/MIRI medium resolution spectroscopy obtained for EX Lup fourteen years after its powerful outburst. EX Lup is now in quiescence and displays a Class II spectrum. We detect a forest of emission lines from molecules previously identified in infrared spectra of classical T Tauri disks: H2O, OH, H2, HCN, C2H2, and CO2. The detection of organic molecules demonstrates that they are back after disappearing during the large outburst. Spectral lines from water and OH are for the first time de-blended and will provide a much improved characterization of their distribution and density in the inner disk. The spectrum also shows broad emission bands from warm, sub-micron size amorphous silicate grains at 10 and 18 um. During the outburst, in 2008, crystalline forsterite grains were annealed in the inner disk within 1 au, but their spectral signatures in the 10 um silicate band later disappeared. With JWST we re-discovered these crystals via their 19.0, 20.0, and 23.5 um emission, whose strength implies that the particles are at ~3 au from the star. This suggests that crystalline grains formed in 2008 were transported outwards and now approach the water snowline, where they may be incorporated into planetesimals. Containing several key tracers of planetesimal and planet formation, EX Lup is an ideal laboratory to study the effects of variable luminosity on the planet-forming material and may provide explanation for the observed high crystalline fraction in solar system comets. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.09140-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.09140) | **An Ice Age JWST inventory of dense molecular cloud ices**  |
|| M. K. McClure, et al. -- incl., <mark>G. Perotti</mark>, <mark>Z.L. Smith</mark>, <mark>W. A. Brown</mark>, <mark>Herma M. Cuppen</mark>, <mark>D.C. Lis</mark>, <mark>H. Linnartz</mark> |
|*Appeared on*| *2023-01-24*|
|*Comments*| *To appear in Nature Astronomy on January 23rd, 2023. 33 pages, 16 figures, 3 tables; includes extended and supplemental data sections. Part of the JWST Ice Age Early Release Science program's science enabling products. Enhanced spectra downloadable on Zenodo at the following DOI: 10.5281/zenodo.7501239*|
|**Abstract**| Icy grain mantles are the main reservoir of the volatile elements that link chemical processes in dark, interstellar clouds with the formation of planets and composition of their atmospheres. The initial ice composition is set in the cold, dense parts of molecular clouds, prior to the onset of star formation. With the exquisite sensitivity of JWST, this critical stage of ice evolution is now accessible for detailed study. Here we show the first results of the Early Release Science program "Ice Age" that reveal the rich composition of these dense cloud ices. Weak ices, including, $^{13}$CO$_2$, OCN$^-$, $^{13}$CO, OCS, and COMs functional groups are now detected along two pre-stellar lines of sight. The $^{12}$CO$_2$ ice profile indicates modest growth of the icy grains. Column densities of the major and minor ice species indicate that ices contribute between 2 and 19% of the bulk budgets of the key C, O, N, and S elements. Our results suggest that the formation of simple and complex molecules could begin early in a water-ice rich environment. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.09621-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.09621) | **The vertical structure of the spiral galaxy NGC 3501: first stages of  the formation of a thin metal-rich disc**  |
|| <mark>Natascha Sattler</mark>, et al. -- incl., <mark>Francesca Pinna</mark>, <mark>Nadine Neumayer</mark> |
|*Appeared on*| *2023-01-24*|
|*Comments*| *15 pages, 12 figures. Accepted for publication in MNRAS*|
|**Abstract**| We trace the evolution of the edge-on spiral galaxy NGC 3501, making use of its stellar populations extracted from deep integral-field spectroscopy MUSE observations. We present stellar kinematic and population maps, as well as the star formation history, of the south-western half of the galaxy. The derived maps of the stellar line-of-sight velocity and velocity dispersion are quite regular, show disc-like rotation, and no other structural component of the galaxy. However, maps of the stellar populations exhibit structures in the mass-weighted and light-weighted age, total metallicity and [Mg/Fe] abundance. These maps indicate that NGC 3501 is a young galaxy, consisting mostly of stars with ages between 2 to 8 Gyr. Also, they show a thicker more extended structure that is metal-poor and $\alpha$-rich, and another inner metal-rich and $\alpha$-poor one with smaller radial extension. While previous studies revealed that NGC 3501 shows only one morphological disc component in its vertical structure, we divided the galaxy into two regions: an inner metal-rich midplane and a metal-poor thicker envelope. Comparing the star formation history of the inner thinner metal-rich disc and the thicker metal-poor disc, we see that the metal-rich component evolved more steadily, while the metal-poor one experienced several bursts of star formation. We propose this spiral galaxy is being observed in an early evolutionary phase, with a thicker disc already in place and an inner thin disc in an early formation stage. So we are probably witnessing the birth of a future massive thin disc, continuously growing embedded in a preexisting thicker disc. |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.09612-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.09612) | **Constraining a Model of the Radio Sky Below 6 MHz Using the Parker Solar  Probe/FIELDS Instrument in Preparation for Upcoming Lunar-based Experiments**  |
|| Neil Bassett, et al. -- incl., <mark>Jack O. Burns</mark> |
|*Appeared on*| *2023-01-24*|
|*Comments*| *18 pages, 10 figures, 5 tables. Under review in the Astrophysical Journal*|
|**Abstract**| We present a Bayesian analysis of data from the FIELDS instrument on board the Parker Solar Probe (PSP) spacecraft with the aim of constraining low frequency ($\lesssim$ 6 MHz) sky in preparation for several upcoming lunar-based experiments. We utilize data recorded during PSP's ``coning roll'' maneuvers, in which the axis of the spacecraft is pointed 45$^{\circ}$ off of the Sun. The spacecraft then rotates about a line between the Sun and the spacecraft with a period of 24 minutes. We reduce the data into two formats: roll-averaged, in which the spectra are averaged over the roll, and phase-binned, in which the spectra are binned according to the phase of the roll. We construct a forward model of the FIELDS observations that includes numerical simulations of the antenna beam, an analytic emissivity function of the galaxy, and estimates of the absorption due to free electrons. Fitting 5 parameters, we find that the roll-averaged data can be fit well by this model and we obtain posterior parameter constraints that are in general agreement with previous estimates. The model is not, however, able to fit the phase-binned data well, likely due to limitations such as the lack of non-smooth emission structure at both small and large scales, enforced symmetry between the northern and southern galactic hemispheres, and large uncertainties in the free electron density. This suggests that significant improvement in the low frequency sky model is needed in order to fully and accurately represent the sky at frequencies below 6 MHz. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.09614-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.09614) | **Point-like sources among z>11 galaxy candidates: contaminants due to  supernovae at high redshifts?**  |
|| Haojing Yan, <mark>Lifan Wang</mark>, Zhiyuan Ma, Lei Hu |
|*Appeared on*| *2023-01-24*|
|*Comments*| **|
|**Abstract**| The recent searches for z>11 galaxies using the James Webb Space Telescope have resulted in an unexpectedly high number of candidate objects, which imply at least an order of magnitude higher number density of $z>11$ galaxies than the previously favored predictions. A question has risen whether there are some new types of contaminants among these candidates. The candidate sample of Yan et al. (2023a), totalling 87 dropouts, is the largest one, and we notice that a number of these candidates are point-like. We hypothesize that the point-source dropouts could be supernovae at high redshifts. Further investigation shows that most of their spectral energy distributions indeed can be explained by supernovae at various redshifts from z ~ 1--15, which lends support to this hypothesis. Attributing such point-source dropouts to supernova contamination cannot eliminate the tension, however, because they only account for ~10% of the Yan et al.'s sample. On the other hand, the discovery of "contaminant" supernovae at $z>3$ will have a series of important implications. Ironically, the existence of supernovae at $z>10$ would still imply that the previously favored picture of early galaxy formation severely underestimates the global star formation rate density such redshifts. Multiple-epoch JWST imaging will be the simplest and yet the most efficient way to further test this hypothesis. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.08744-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.08744) | **PODIUM:A Pulsar Navigation Unit for Science Missions**  |
|| Francesco Cacciatore, et al. -- incl., <mark>Pablo Hermosín</mark>, <mark>Margarita Hernanz</mark> |
|*Appeared on*| *2023-01-24*|
|*Comments*| **|
|**Abstract**| PODIUM is a compact spacecraft navigation unit, currently being designed to provide interplanetary missions with autonomous position and velocity estimations. The unit will make use of Pulsar X-ray observations to measure the distance and distance rate from the host spacecraft to the Solar System Barycenter. Such measurements will then be used by the onboard orbit determination function to estimate the complete orbital elements of the spacecraft. The design aims at 6 kg of mass and 20 W of power, in a volume of 150 mm by 240 mm by 600 mm. PODIUM is designed to minimize the impact on the mission operational and accommodation constraints. The architecture is based on a grazing incidence X-ray telescope with focal distance limited to 50 cm. The effective area shall be in the range 25 to 50 cm2 for photon energies in the range 0.2-10 keV, requiring nesting of several mirrors in the Wolter-1 geometry. Grazing incidence angles will be very small, below 2 deg. The current target FOV is 0.25 deg. The pulsars photon arrivals are detected with a single pixel Silicon Drift Detector (SDD) sensor with timing accuracy below 1usec. The unit has no gimbaling to meet the applicable power, size and mass requirements. Instead, the host spacecraft shall slew and point to allow pulsar observation. The avionics architecture is based on a radiation hardened LEON4 processor, to allow a synchronous propagation task and measurement generation and orbit determination step in an asynchronous task. PODIUM will enable higher autonomy and lower cost for interplanetary missions. L2 space observatories and planetary flybys are the current reference use cases. Onboard autonomous state estimation can reduce the ground support effort required for navigation and orbit correction/maintenance computation, and reduce the turnaround time, thus enabling more accurate maneuvers, reducing the orbit maintenance mass budget. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.08946-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.08946) | **Heating or Cooling: Study of Advective Heat Transport in the Inflow and  the Outflow of Optically Thin Advection-dominated Accretion Flows**  |
|| <mark>Cheng-Liang Jiao</mark> |
|*Appeared on*| *2023-01-24*|
|*Comments*| *9 pages, 4 figures, accepted by ApJ*|
|**Abstract**| Advection is believed to be the dominant cooling mechanism in optically thin advection-dominated accretion flows (ADAF's). When outflow is considered, however, the first impression is that advection should be of opposite sign in the inflow and the outflow, due to the opposite direction of radial motion. Then how is the energy balance achieved simultaneously? We investigate the problem in this paper, analysing the profiles of different components of advection with self-similar solutions of ADAF's in spherical coordinates ($r\theta\phi$). We find that for $n < 3\gamma/2-1$, where $n$ is the density index in $\rho \propto r^{-n}$ and $\gamma$ is the heat capacity ratio, the radial advection is a heating mechanism in the inflow and a cooling mechanism in the outflow. It becomes 0 for $n = 3\gamma/2-1$, and turns to a cooling mechanism in the inflow and a heating mechanism in the outflow for $n > 3\gamma/2-1$. The energy conservation is only achieved when the latitudinal ($\theta$-direction) advection is considered, which takes an appropriate value to maintain energy balance, so that the overall effect of advection, no matter the parameter choices, is always a cooling mechanism that cancels out the viscous heating everywhere. For the extreme case of $n=3/2$, latitudinal motion stops, viscous heating is balanced solely by radial advection, and no outflow is developed. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2301.09102-b31b1b.svg)](https://arxiv.org/abs/arXiv:2301.09102) | **Proton and electron irradiations of CH4:H2O mixed ices**  |
|| Duncan V. Mifsud, et al. -- incl., <mark>Péter Herczku</mark> |
|*Appeared on*| *2023-01-24*|
|*Comments*| *Published as an open access article in the MDPI journal Atoms*|
|**Abstract**| The organic chemistry occurring in interstellar environments may lead to the production of complex molecules that are relevant to the emergence of life. Therefore, in order to understand the origins of life itself, it is necessary to probe the chemistry of carbon-bearing molecules under conditions that simulate interstellar space. Several of these regions, such as dense molecular cores, are exposed to ionizing radiation in the form of galactic cosmic rays, which may act as an important driver of molecular destruction and synthesis. In this paper, we report the results of a comparative and systematic study of the irradiation of CH4:H2O ice mixtures by 1 MeV protons and 2 keV electrons at 20 K.We demonstrate that our irradiations result in the formation of a number of new products, including both simple and complex daughter molecules such as C2H6, C3H8, C2H2, CH3OH, CO, CO2, and probably also H2CO. A comparison of the different irradiation regimes has also revealed that proton irradiation resulted in a greater abundance of radiolytic daughter molecules compared to electron irradiation, despite a lower radiation dose having been administered. These results are important in the context of the radiation astrochemistry occurring within the molecular cores of dense interstellar clouds, as well as on outer Solar System objects. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2301.08770.md
    + _build/html/tmp_2301.08770/./fig_dust19b.png
    + _build/html/tmp_2301.08770/./image.png
    + _build/html/tmp_2301.08770/./test_aper_2.png
exported in  _build/html/2301.09140.md
    + _build/html/tmp_2301.09140/./main_fig1.png
    + _build/html/tmp_2301.09140/./main_fig4_top.png
    + _build/html/tmp_2301.09140/./main_fig4_bottom.png
    + _build/html/tmp_2301.09140/./main_fig2.png
exported in  _build/html/2301.09621.md
    + _build/html/tmp_2301.09621/./plots/pop_lw.png
    + _build/html/tmp_2301.09621/./plots/pop_mw.png
    + _build/html/tmp_2301.09621/./plots/V_sigma.png
    + _build/html/tmp_2301.09621/./plots/h3_h4.png
    + _build/html/tmp_2301.09621/./plots/met.png
    + _build/html/tmp_2301.09621/./plots/alpha.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# JWST/MIRI Spectroscopy of the Disk of the Young Eruptive Star EX Lup in Quiescence

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.08770-b31b1b.svg)](https://arxiv.org/abs/2301.08770)<mark>Appeared on: 2023-01-24</mark> - _9 pages, 4 figures, accepted for publication in ApJL_

</div>
<div id="authors">

\'Agnes K\'osp\'al, et al. -- incl., <mark>Lindsey Diehl</mark>, <mark>Jeroen Bouwman</mark>, <mark>Joel D. Green</mark>, <mark>Thomas Henning</mark>

</div>
<div id="abstract">

**Abstract:** EX Lup is a low-mass pre-main sequence star that occasionally shows accretion-related outbursts. Here, we present JWST/MIRI medium resolution spectroscopy obtained for EX Lup fourteen years after its powerful outburst. EX Lup is now in quiescence and displays a Class II spectrum. We detect a forest of emission lines from molecules previously identified in infrared spectra of classical T Tauri disks:\ce{H2O}, OH,\ce{H2}, HCN,\ce{C2H2}, and\ce{CO2}. The detection of organic molecules demonstrates that they are back after disappearing during the large outburst. Spectral lines from water and OH are for the first time de-blended and will provide a much improved characterization of their distribution and density in the inner disk. The spectrum also shows broad emission bands from warm, sub-micron size amorphous silicate grains at 10 and 18$ \mu$m. During the outburst, in 2008, crystalline forsterite grains were annealed in the inner disk within 1 au, but their spectral signatures in the 10$\mu$m silicate band later disappeared. With JWST we re-discovered these crystals via their 19.0, 20.0, and 23.5$ \mu$m emission, whose strength implies that the particles are at$\sim$3 au from the star. This suggests that crystalline grains formed in 2008 were transported outwards and now approach the water snowline, where they may be incorporated into planetesimals. Containing several key tracers of planetesimal and planet formation, EX Lup is an ideal laboratory to study the effects of variable luminosity on the planet-forming material and may provide explanation for the observed high crystalline fraction in solar system comets.

</div>

<div id="div_fig1">

<img src="tmp_2301.08770/./fig_dust19b.png" alt="Fig2" width="100%"/>

**Figure 2. -** Continuum-subtracted JWST/MIRI spectrum of EX Lup with various dust models, and with the Spitzer/IRS quiescent spectrum from 2005. The model curves were shifted for clarity.\label{fig:dust} (*fig:dust*)

</div>
<div id="div_fig2">

<img src="tmp_2301.08770/./image.png" alt="Fig3" width="100%"/>

**Figure 3. -** Continuum-subtracted MIRI spectrum with slab models for the identification of molecular lines.\label{fig:slab_models} (*fig:slab_models*)

</div>
<div id="div_fig3">

<img src="tmp_2301.08770/./test_aper_2.png" alt="Fig1" width="100%"/>

**Figure 1. -** JWST/MIRI MRS spectrum of EX Lup (color), shown together with an earlier Spitzer/IRS spectrum (gray), both taken in quiescence. The different colors mark the individual sub-bands of the JWST spectrum. We indicated some of the observed molecular and solid state features. We note that the spectrum contains hundreds of \ce{H2O} lines; we only marked a few of them as an illustration. The JWST spectrum shown here is available as Data behind the Figure.\label{fig:test_aper} (*fig:test_aper*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\bibinfo}[2]{#2}$
$\newcommand{\eprint}[2][]{\url{#2}}$
$\newcommand{\tablenotes}$
$\newcommand{\cmark}{\textcolor{green}{\ding{51}}}$
$\newcommand{\xmark}{\textcolor{red}{\ding{55}}}$
$\newcommand{\ED}[1]{{\color{blue!50}\;ED: #1}}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.0}$
$\newcommand{\}{url}$
$\newcommand{\urlprefix}{URL }$
$\newcommand{\doiprefix}{DOI: }$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\bibinfo$}[2]{#2}$
$\newcommand{$\eprint$}[2][]{\url{#2}}$
$\newcommand{$\tablenotes$}$
$\newcommand{$\cmark$}{\textcolor{green}{\ding{51}}}$
$\newcommand{$\xmark$}{\textcolor{red}{\ding{55}}}$
$\newcommand{$\ED$}[1]{{\color{blue!50}\;ED: #1}}$
$\newcommand{$\arraystretch$}{1.3}$
$\newcommand{$\arraystretch$}{1.0}$
$\newcommand{\}{url}$
$\newcommand{$\urlprefix$}{URL }$
$\newcommand{$\doiprefix$}{DOI: }$</div>



<div id="title">

# An Ice Age JWST inventory of dense molecular cloud ices

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.09140-b31b1b.svg)](https://arxiv.org/abs/2301.09140)<mark>Appeared on: 2023-01-24</mark> - _To appear in Nature Astronomy on January 23rd, 2023. 33 pages, 16 figures, 3 tables; includes extended and supplemental data sections. Part of the JWST Ice Age Early Release Science program's science enabling products. Enhanced spectra downloadable on Zenodo at the following DOI: 10.5281/zenodo.7501239_

</div>
<div id="authors">

M. K. McClure, et al. -- incl., <mark>G. Perotti</mark>, <mark>Z.L. Smith</mark>, <mark>W. A. Brown</mark>, <mark>Herma M. Cuppen</mark>, <mark>D.C. Lis</mark>, <mark>H.~Linnartz</mark>

</div>
<div id="abstract">

**Abstract:** \bf{Icy grain mantles are the main reservoir of the volatile elements that link chemical processes in dark, interstellar clouds with the formation of planets and composition of their atmospheres. The initial ice composition is set in the cold, dense parts of molecular clouds, prior to the onset of star formation. With the exquisite sensitivity of JWST, this critical stage of ice evolution is now accessible for detailed study. Here we show the first results of the Early Release Science program "Ice Age" that reveal the rich composition of these dense cloud ices. Weak ices, including, $^{13}$CO$_2$, OCN$^-$, $^{13}$CO, OCS, and COMs functional groups are now detected along two pre-stellar lines of sight. The $^{12}$CO$_2$ ice profile indicates modest growth of the icy grains. Column densities of the major and minor ice species indicate that ices contribute between 2 and 19\% of the bulk budgets of the key C, O, N, and S elements. Our results suggest that the formation of simple and complex molecules could begin early in a water-ice rich environment.}

</div>

<div id="div_fig1">

<img src="tmp_2301.09140/./main_fig1.png" alt="Fig1" width="100%"/>

**Figure 1. -** \textbf{NIRSpec FS [NIRCam WFSS] and MIRI LRS spectra of NIR38 and J110621.} Top: Full NIRSpec FS and MIRI LRS spectra of NIR38
($A_V\sim60$, solid navy line) and J110621 ($A_V\sim95$, solid light gray line), with associated continuum fits (dotted lines). For NIR38, a preliminary NIRCam WFSS spectrum has been scaled to the NIRSpec spectrum at 3.8 $\mu$m and spliced in to cover the NIRSpec FS gap from 3.85--3.9 $\mu$m and extend the spectrum to 2.5 $\mu$m. Ice absorption features are color-coded according to species and labelled in the NIR38 spectrum. Wavelength regions used for the continuum fit are indicated by light gray bars (NIRSpec) and dark gray filled circles (MIRI) at the bottom of the top panel. Bottom: Zoom in on the weaker ice features and structure revealed by JWST. The potential dangling O-H feature is indicted by "dO-H", and the combination modes of CO$_2$ and H$_2$O by "combi." (*main_fig1*)

</div>
<div id="div_fig2">

<img src="tmp_2301.09140/./main_fig4_top.png" alt="Fig4.1" width="50%"/><img src="tmp_2301.09140/./main_fig4_bottom.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** \textbf{Barplots showing the derived ice column density for different species  towards NIR38 ($A_V \sim 60$ mag) and J110621 ($A_V \sim 95$ mag).}(Top) Column densities of the ice species identified in this work, compared to the literature values of Elias 16 ($A_V \sim 19$ mag)\cite{Knez2005}. The column densities of the major ice components are from the global \texttt{ENIIGMA} fit (best of n=112 models), and we use the values from the local fits for the minor ice components. Black arrows indicate upper limits and error bars are taken from the 3$\sigma$ confidence intervals. (Bottom) Relative column densities of the detected ices, normalized to H$_2$O ice. For the major ice components, we use the values from the global \texttt{ENIIGMA} fit, and for the minor ice components, we use the values from the local fits. Black arrows indicate upper limits and error bars are taken from the 3$\sigma$ confidence intervals. (*main_fig4*)

</div>
<div id="div_fig3">

<img src="tmp_2301.09140/./main_fig2.png" alt="Fig2" width="100%"/>

**Figure 2. -** \textbf{Data quality comparison for NIR38 and J110621.}(Top panel) Comparison of the NIRCam WFSS (blue), NIRSpec FS (black), and MIRI LRS FS (red) spectra of the $A_V=60$ background star. Error bars (gray) are 3$\sigma$, and in some regions are smaller than the thickness of the lines. Spitzer IRAC photometry (gold points) from the IPAC SEIP catalog is given for reference, with error bars and bandpass indicated. (Bottom panel) Comparison of NIRSpec FS and MIRI LRS FS data for the $A_V=95$ star. Colors are the same as in the top panel. (*main_fig2*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\thebibliography$}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The vertical structure of the spiral galaxy NGC 3501: first stages of the formation of a thin metal-rich disc

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2301.09621-b31b1b.svg)](https://arxiv.org/abs/2301.09621)<mark>Appeared on: 2023-01-24</mark> - _15 pages, 12 figures. Accepted for publication in MNRAS_

</div>
<div id="authors">

<mark><mark>Natascha Sattler</mark></mark>, et al. -- incl., <mark><mark>Francesca Pinna</mark></mark>, <mark><mark>Nadine Neumayer</mark></mark>

</div>
<div id="abstract">

**Abstract:** We trace the evolution of the edge-on spiral galaxy NGC 3501, making use of its stellar populations extracted from deep integral-field spectroscopy MUSE observations. We present stellar kinematic and population maps, as well as the star formation history, of the south-western half of the galaxy.The derived maps of the stellar line-of-sight velocity and velocity dispersion are quite regular, show disc-like rotation, and no other structural component of the galaxy.However, maps of the stellar populations exhibit structures in the mass-weighted and light-weighted age, total metallicity and[Mg/Fe]abundance.These maps indicate that NGC 3501 is a young galaxy, consisting mostly of stars with ages between 2 to 8 Gyr.Also, they show a thicker more extended structure that is metal-poor and$\alpha$-rich, and another inner metal-rich and$\alpha$-poor one with smaller radial extension.While previous studies revealed that NGC 3501 shows only one morphological disc component in its vertical structure, we divided the galaxy into two regions: an inner metal-rich midplane and a metal-poor thicker envelope.Comparing the star formation history of the inner thinner metal-rich disc and the thicker metal-poor disc,we see that the metal-rich component evolved more steadily, while the metal-poor one experienced several bursts of star formation.We propose this spiral galaxy is being observed in an early evolutionary phase, with a thicker disc already in place and an inner thin disc in an early formation stage. So we are probably witnessing the birth of a future massive thin disc, continuously growing embedded in a preexisting thicker disc.\

</div>

<div id="div_fig1">

<img src="tmp_2301.09621/./plots/pop_lw.png" alt="Fig7.1" width="50%"/><img src="tmp_2301.09621/./plots/pop_mw.png" alt="Fig7.2" width="50%"/>

**Figure 7. -** Left: Stellar population maps showing the light-weighted age (top), total metallicity [M/H](middle) and [Mg/Fe]-abundance (bottom). Right: Stellar population maps showing the mass-weighted age (top), total metallicity [M/H](middle) and [Mg/Fe]-abundance (bottom). The approximate positions of the MUSE pointings are indicated by grey lines and the discarded bins are displayed in grey, while the isophotes are displayed in black.
     (*fig:pop*)

</div>
<div id="div_fig2">

<img src="tmp_2301.09621/./plots/V_sigma.png" alt="Fig6.1" width="50%"/><img src="tmp_2301.09621/./plots/h3_h4.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Left: Stellar kinematic maps of the mean velocity $V$(upper panel) and velocity dispersion $\sigma$(bottom panel) of the stellar LOSVD. Right: Kinematic maps show values of $h_3$ and $h_4$ as results from the stellar kinematic pPXF fitting. The discarded bins are plotted in grey and the approximate positions of the MUSE pointings are indicated by grey lines, while the isophotes are displayed in black.
     (*fig:kin*)

</div>
<div id="div_fig3">

<img src="tmp_2301.09621/./plots/met.png" alt="Fig8.1" width="50%"/><img src="tmp_2301.09621/./plots/alpha.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** Star formation history in terms of the mean mass fraction (normalized to the galaxy mass in the region covered by our data) in the metal-poor (bottom) and metal-rich (top) components as a function of age. Age bins are color-coded by metallicity [M/H](left), and by [Mg/Fe](right). Black dashed edges mark age bins with [Mg/Fe] abundances that we do not consider totally reliable, because of difficulties in retrieving the [$\alpha$/Fe] for young stars as discussed in Appendix \ref{appendix:system_unc}.
     (*fig:metalpha*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

180  publications in the last 7 days.
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/html/2301.06575.md
	 _build/html/2301.05720.md
	 _build/html/2301.05718.md
	 _build/html/2301.05034.md
	 _build/html/2301.04656.md
	 _build/html/2301.04533.md
	 _build/html/2301.04442.md
	 _build/html/2301.03811.md
	 _build/html/2301.03689.md
	 _build/html/2301.03387.md
	 _build/html/2301.03209.md
	 _build/html/2301.02671.md
	 _build/html/2301.02477.md
	 _build/html/2301.02313.md
	 _build/html/2301.02265.md
	 _build/html/2301.01937.md
	 _build/html/2301.01895.md
	 _build/html/2301.01684.md
	 _build/html/2301.01306.md
	 _build/html/2301.00881.md
	 _build/html/2301.00854.md
	 _build/html/2301.00578.md
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers